In [76]:
!pip install piq

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [77]:
import torch
import torch.nn.functional as F
import random
import numpy as np
from torch import nn

class Dummy:
    '''
    Creating a dummy class to maintain consistency in function or method calls 
    when certain classes are None, useful in simplifying code logic
    '''
    def __init__(self):
        pass
    def forward(self, x):
        return x

class DI:
    def __init__(self, low, high, p:float=0.5, transform=nn.Identity) -> None:
        self.low = low
        self.high = high
        self.p = p
        self.transform = transform

    def forward(self, x):
        if self.p < random.random():
            return x
        # Random resize
        rnd = torch.randint(low=self.low, high=self.high, size=(1, ))
        rescaled = F.interpolate(x, size=(rnd, rnd), mode='bilinear')
        # Calculate padding
        h_rem = self.high - rnd
        w_rem = self.high - rnd
        pad_top = random.randint(0, h_rem)
        pad_bottom = h_rem - pad_top
        pad_left = random.randint(0, w_rem)
        pad_right = w_rem - pad_left
        # Apply padding
        padded = F.pad(rescaled, (pad_left, pad_right, pad_top, pad_bottom), value=0)
        return self.transform(padded)

class TI:
    def __init__(self, 
        kernel_size, 
        nsig,
        device="cuda"
        ) -> None:
        import scipy.stats as st
        kern1d = st.norm.pdf(np.linspace(-nsig, nsig, kernel_size))
        kernel_raw = np.outer(kern1d, kern1d)
        kernel = kernel_raw / kernel_raw.sum()
        stack_kernel = np.stack([kernel, kernel, kernel])
        stack_kernel = np.expand_dims(stack_kernel, 1)
        self.kernel = torch.from_numpy(stack_kernel).float().to(device)

    def forward(self, grad):
        return F.conv2d(grad, self.kernel, stride=1, padding="same", groups=3)

class Admix:
    '''
    Paper : Admix: Enhancing the Transferability of Adversarial Attacks
    '''
    def __init__(self, 
                gamma=[1 / (2 ** i) for i in range(5)], 
                eta=0.2, 
                num_samples=3
                ) -> None:
        self.eta = eta
        self.gamma = gamma
        assert num_samples >= 1
        self.num_samples = num_samples
        if not isinstance(self.gamma, list):
            self.m = 1
        else:
            self.m = len(self.gamma)

    def forward(self, x):
        batch_size = x.size(0)
        idxs = torch.arange(batch_size)
        x_admix = []
        t = x.clone()
        if self.eta != 0:
            for _ in range(self.num_samples):
                shuffled_idxs = idxs[torch.randperm(batch_size)]
                slave = x[shuffled_idxs]
                x_admix.append(x + self.eta * slave)
            x_admix = torch.cat(x_admix, dim=0)
        else:
            x_admix = t
        return torch.cat([x_admix * gamma for gamma in self.gamma], dim=0) if self.gamma else x_admix

class VT:
    '''
    Paper : "Enhancing the Transferability of Adversarial Attacks through Variance Tuning"
    '''
    def __init__(self, 
        num_samples,
        bound,
        dist,
        admix=None, 
        dim=None,
        ) -> None:
        self.num_samples = num_samples
        self.bound = bound
        self.dim = dim
        self.admix = admix
        self.dist = dist
        if not isinstance(admix, Dummy):
            self.m = self.admix.m
            self.n = self.admix.num_samples
            self.gamma = self.admix.gamma
        else:
            self.m = self.n = 1

    def forward(self, x, model, labels, target_labels=None, device="cuda"): 
        if self.dist == "uniform":
            perturbations = (torch.rand((self.num_samples, *x.shape), device=device) * 2 - 1) * self.bound.to(device)
        elif self.dist == "normal":
            perturbations = torch.randn((self.num_samples, *x.shape), device=device) * self.bound.to(device)
        x_neighbors = x.unsqueeze(0) + perturbations  # Shape: (self.num_samples, batch_size, *x.shape[1:])
        x_neighbors = x_neighbors.view(-1, *x.shape[1:])  # Flatten for batch processing
        
        x_neighbors = self.admix.forward(x_neighbors)
        x_neighbors.retain_grad()
    
        outputs = model(self.dim.forward(x_neighbors))  # Outputs for all perturbed samples
    
        loss_fn = nn.CrossEntropyLoss()  # No reduction for per-sample loss
        if target_labels is not None:
            losses = -loss_fn(outputs, target_labels)
        else:
            losses = loss_fn(outputs, labels)
    
        grads = torch.autograd.grad(losses, x_neighbors, retain_graph=False, create_graph=False)[0]
        grads = grads.mean(dim=0)
    
        return grads

In [78]:
from torch import nn
import torch
from abc import ABC, abstractmethod
import math



class Attack(ABC):
    def __init__(self, 
                alpha, 
                eps, 
                mean=None, std=None,
                n_iter=10,
                device="cuda",
                activation:str="sign",
                dim=None,
                tim=None,
                admix=None,
                vt=None,
                ) -> None:
        self.device = device
        if mean is not None:
            self.mean = torch.tensor(mean).view(1, 3, 1, 1).to(self.device)
        else:
            self.mean = torch.tensor([0, 0, 0]).view(1, 3, 1, 1).to(self.device)
        if std is not None:
            self.std = torch.tensor(std).view(1, 3, 1, 1).to(self.device)
            adjustment = [(1 / 255) / s for s in std]
        else:
            self.std = torch.tensor([1, 1, 1]).view(1, 3, 1, 1).to(self.device)
            adjustment = [1, 1, 1]

        self.ad_alpha = alpha * torch.tensor(adjustment, device=device).view(1, 3, 1, 1)
        self.ad_eps = eps * torch.tensor(adjustment, device=device).view(1, 3, 1, 1)
        self.n_iter = n_iter
        self.perturbation = None
        if activation not in ACTIVATION:
            raise NotImplementedError(f"Please implement {activation} function in activation.py")

        self.activation_name = activation
        self.activation = ACTIVATION[activation]
        self.grad = None

        self.dim = DI(**dim) if dim is not None else Dummy()
        self.tim = TI(**tim) if tim is not None else Dummy()
        self.admix = Dummy()
        self.vt = Dummy()

        if admix is not None:
            self.admix = Admix(**admix)
            self.m = self.admix.m
            self.n = self.admix.num_samples
            self.gamma = self.admix.gamma
        else:
            self.m = self.n = 1

        if vt is not None:
            self.vt = VT(vt["num_samples"], (self.ad_eps * vt["bound"]).view(1, 3, 1, 1).to(device), 
                        vt["dist"],
                        admix=self.admix, dim=self.dim)
            self.N = self.vt.num_samples
        else:
            self.N = 1

    @abstractmethod
    def init_components(self):
        pass

    def nesterov(self, adv, images):
        return adv

    def forward(self, model, images, labels, target_labels=None):
        early_stop = False      

        labels_ = labels.repeat(self.m * self.n).to(self.device)
        
        images = images.to(self.device)
        if target_labels is not None:
            target_labels_ = target_labels.repeat(self.m * self.n).to(self.device)
            vt_target_labels = target_labels_.repeat(self.N).to(self.device)

        
        vt_labels = labels_.repeat(self.N).to(self.device)
        loss = nn.CrossEntropyLoss()
        v_grad = torch.zeros_like(images).to(self.device)
        self.perturbation = torch.zeros_like(images).to(self.device)
        self.init_components(images)
        adv = images.detach().clone().to(self.device)
        for idx in range(self.n_iter):
            adv.requires_grad = True
            # Initialize NAG
            adv_nes = self.nesterov(adv, images)
            adv_ = self.admix.forward(adv_nes)
            adv__ = self.dim.forward(adv_)
            outputs = model(adv__)
            l = -loss(outputs, target_labels_) if target_labels is not None else loss(outputs, labels_)
            if early_stop:
                if (target_labels is not None and outputs.max(1)[1] == target_labels_) or (target_labels is None and outputs.max(1)[1] != labels_):
                    return adv_nes.detach(), outputs.max(1)[1], l.detach().item(), self.perturbation.detach(), idx + 1
            noise = torch.autograd.grad(
                l, adv, retain_graph=False, create_graph=False
            )[0]
            if isinstance(self.admix, Admix):
                noise = torch.sum(noise, dim=0) / (self.m * self.n)
            self.grad = noise + v_grad
            self.grad = self.tim.forward(self.grad)
            self.grad = self.grad / torch.mean(torch.abs(self.grad), dim=(1, 2, 3), keepdim=True)
            if isinstance(self.vt, VT):
                v_grad = self.vt.forward(adv_nes, model, vt_labels, vt_target_labels if target_labels is not None else target_labels, self.device) - noise
            adv = self.update(adv, idx)
            adv = self.clip(adv, images).detach()
        return adv.detach(), model(adv).max(1)[1], l.mean().detach().item(), self.perturbation.detach(), self.n_iter 

    @abstractmethod
    def update(self, adv, idx):
        pass

    def clip(self, adv, images):
        delta = torch.clamp(adv - images, min=-self.ad_eps, max=self.ad_eps)
        self.perturbation = delta
        adv_ = images + delta
        adv_ = adv_ * self.std + self.mean        
        adv_ = torch.clamp(adv_, 0, 1)
        adv_ = (adv_ - self.mean) / self.std
        return adv_

class Momentum(Attack):
    def __init__(self, alpha=1, eps=5, mean=None, std=None,  n_iter=10, device="cuda", 
                beta=1.0,
                activation="sign", 
                dim:DI=None, tim:TI=None, admix:Admix=None, vt:VT=None
                ):
        super().__init__(alpha, eps, mean, std, n_iter, device, activation, dim, tim, admix, vt)
        self.beta = beta
        self.momentum = None

    def init_components(self, x):
        self.momentum = torch.zeros_like(x).to(self.device)

    def update(self, adv, idx):
        g = self.beta * self.momentum + self.grad
        self.momentum = g
        return adv + self.ad_alpha * self.activation(g)

class Nesterov(Momentum):
    def __init__(self, alpha=1, eps=5, mean=None, std=None, n_iter=10, device="cuda", 
                beta=1.0,
                activation="sign",
                dim:DI=None, tim:TI=None, admix:Admix=None, vt:VT=None
                ) -> None:
        super().__init__(alpha, eps, mean, std, n_iter, device, beta, activation, dim, tim, admix, vt)

    def nesterov(self, adv, images):
        adv_nes = adv + self.beta * self.ad_alpha * self.activation(self.momentum)
        return self.clip(adv_nes, images)

class Adam(Attack):
    def __init__(self, alpha=1, eps=5, mean=None, std=None, n_iter=10, device="cuda",
                beta_1=0.9, beta_2=0.999, delta=1e-8,
                activation="softsign",
                dim:DI=None, tim:TI=None, admix:Admix=None, vt:VT=None
                ) -> None:
        super().__init__(alpha, eps, mean, std, n_iter, device, activation, dim, tim, admix, vt)
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.delta = delta
        self.g = None

    def init_components(self, x):
        self.g = torch.zeros_like(x).to(self.device)
        self.momentum_1 = torch.zeros_like(x).to(self.device)
        self.momentum_2 = torch.zeros_like(x).to(self.device)

    def update(self, adv, idx):
        self.momentum_1 = self.beta_1 * self.momentum_1 + (1 - self.beta_1) * self.grad
        self.momentum_2 = self.beta_2 * self.momentum_2 + (1 - self.beta_2) * self.grad ** 2
        b_momentum_1 = self.momentum_1 / (1 - self.beta_1 ** (idx + 1))
        b_momentum_2 = self.momentum_2 / (1 - self.beta_2 ** (idx + 1))
        self.g = (b_momentum_1 / (torch.sqrt(b_momentum_2) + self.delta))
        return adv + self.ad_alpha * self.activation(self.g)

class AdaBelief(Adam):
    def __init__(self, alpha=1, eps=5, mean=None, std=None, n_iter=10, device="cuda",
                beta_1=0.9, beta_2=0.999, delta=1e-8,
                activation="softsign",
                dim:DI=None, tim:TI=None, admix:Admix=None, vt:VT=None
                ) -> None:
        super().__init__(alpha, eps, mean, std, n_iter, device, beta_1, beta_2, delta, activation, dim, tim, admix, vt)

    def update(self, adv, idx):
        self.momentum_1 = self.beta_1 * self.momentum_1 + (1 - self.beta_1) * self.grad
        self.momentum_2 = self.beta_2 * self.momentum_2 + (1 - self.beta_2) * (self.grad - self.momentum_1) ** 2 + self.delta
        b_momentum_1 = self.momentum_1 / (1 - self.beta_1 ** (idx + 1))
        b_momentum_2 = self.momentum_2 / (1 - self.beta_2 ** (idx + 1))
        self.g = (b_momentum_1 / (torch.sqrt(b_momentum_2) + self.delta))
        adv = adv + self.ad_alpha * self.activation(self.g)
        return adv

class NAdam(Adam):
    def __init__(self, alpha=1, eps=5, mean=None, std=None, n_iter=10, device="cuda",
                beta_1=0.9, beta_2=0.999, delta=1e-8,
                activation="softsign",
                dim:DI=None, tim:TI=None, admix:Admix=None, vt:VT=None
                ) -> None:
        super().__init__(alpha, eps, mean, std, n_iter, device, beta_1, beta_2, delta, activation, dim, tim, admix, vt)
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.delta = delta

    def update(self, adv, idx):
        self.momentum_1 = self.beta_1 * self.momentum_1 + (1 - self.beta_1) * self.grad
        self.momentum_2 = self.beta_2 * self.momentum_2 + (1 - self.beta_2) * self.grad ** 2
        b_momentum_1 = self.momentum_1 / (1 - self.beta_1 ** (idx + 1))
        b_momentum_2 = self.momentum_2 / (1 - self.beta_2 ** (idx + 1))
        self.g = (self.beta_1 * b_momentum_1 + (1 - self.beta_1) * self.grad / (1 - self.beta_1 ** (idx + 1))) / (torch.sqrt(b_momentum_2) + self.delta)
        adv = adv + self.ad_alpha * self.activation(self.g)
        return adv

# Dataset class

In [79]:
from torch.utils.data import Dataset
import os
import pandas as pd
import tqdm, imageio, tifffile
from PIL import Image
import numpy as np



class ImageCSVDataset(Dataset):
    '''
    return: path, image, label, target_label
    '''
    def __init__(self, csv_file, image_dir, num=846, transform=None, decrypt=None, mode="validate"):
        self.data = pd.read_csv(csv_file)[:num] if mode == "validate" else pd.read_csv(csv_file)[num:800]
        self.image_dir = image_dir
        self.transform = transform
        if decrypt is None:
            self.format = ".png"
        else:
            self.format = ".tiff"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.data.iloc[idx]['ImageId']) + self.format
        if self.format == ".png":
            image = Image.open(image_path).convert('RGB')  # Assuming RGB images
        else:
            image = imageio.imread(image_path)
        if self.transform:
            image = self.transform(image)
        label = self.data.iloc[idx]['TrueLabel'] - 1
        target_label = self.data.iloc[idx]['TargetClass'] - 1
        return image_path, image, label, target_label

# Configuration of Hyperparameters

In [80]:
import torchvision.models as models

low = 224
high = 330

low32 = low + 32
low64 = low + 64
low1_1 = int(low * 1.1)
low1_2 = int(low * 1.2)
low1_3 = int(low * 1.3)
low1_4 = int(low * 1.4)
low1_5 = int(low * 1.5)

MOM_DIM = {
    "convnext_small": {"low": low, "high": low32, "p": 0.5},
    "maxvit_t": {"low": low, "high": low32, "p": 0.5},
    "resnet152": {"low": low, "high": low64, "p": 0.5},
    "resnext101_64x4d": {"low": low, "high": low64, "p": 0.5},
    "swin_v2_s": {"low": low, "high": low32, "p": 0.5},
    "vit_l_16": {"low": low, "high": low32, "p": 0.5},
    "wide_resnet101_2": {"low": low, "high": low64, "p": 0.5},
}
NES_DIM = {
    "convnext_small": {"low": low, "high": low32, "p": 0.5},
    "maxvit_t": {"low": low, "high": low32, "p": 0.5},
    "resnet152": {"low": low, "high": low32, "p": 0.5},
    "resnext101_64x4d": {"low": low, "high": low64, "p": 0.5},
    "swin_v2_s": {"low": low, "high": low32, "p": 0.5},
    "vit_l_16": {"low": low, "high": low32, "p": 0.5},
    "wide_resnet101_2": {"low": low, "high": low64, "p": 0.5},
}
ADAM_DIM = {    
    "convnext_small": {"low": low, "high": low32, "p": 0.5},
    "maxvit_t": {"low": low, "high": low64, "p": 0.5},
    "resnet152": {"low": low, "high": low1_1, "p": 0.5},
    "resnext101_64x4d": {"low": low, "high": low64, "p": 0.5},
    "swin_v2_s": {"low": low, "high": low32, "p": 0.5},
    "vit_l_16": {"low": low, "high": low32, "p": 0.5},
    "wide_resnet101_2": {"low": low, "high": low64, "p": 0.5},
}
NADAM_DIM = {
    "convnext_small": {"low": low, "high": low32, "p": 0.5},
    "maxvit_t": {"low": low, "high": low32, "p": 0.5},
    "resnet152": {"low": low, "high": low1_1, "p": 0.5},
    "resnext101_64x4d": {"low": low, "high": low1_1, "p": 0.5},
    "swin_v2_s": {"low": low, "high": low32, "p": 0.5},
    "vit_l_16": {"low": low, "high": low32, "p": 0.5},
    "wide_resnet101_2": {"low": low, "high": low64, "p": 0.5},
}
ADABELIEF_DIM = {
    "convnext_small": {"low": low, "high": low32, "p": 0.5},
    "maxvit_t": {"low": low, "high": low32, "p": 0.5},
    "resnet152": {"low": low, "high": low1_1, "p": 0.5},
    "resnext101_64x4d": {"low": low, "high": low1_3, "p": 0.5},
    "swin_v2_s": {"low": low, "high": low32, "p": 0.5},
    "vit_l_16": {"low": low, "high": low32, "p": 0.5},
    "wide_resnet101_2": {"low": low, "high": low64, "p": 0.5},
}

for k in MOM_DIM.keys():
    MOM_DIM[k]["transform"] = models.get_model_weights(k).DEFAULT.transforms()
    NES_DIM[k]["transform"] = models.get_model_weights(k).DEFAULT.transforms()
    ADAM_DIM[k]["transform"] = models.get_model_weights(k).DEFAULT.transforms()
    NADAM_DIM[k]["transform"] = models.get_model_weights(k).DEFAULT.transforms()
    ADABELIEF_DIM[k]["transform"] = models.get_model_weights(k).DEFAULT.transforms()

MOM_TIM = {
    "convnext_small": {"kernel_size": 15, "nsig": 3},
    "maxvit_t": {"kernel_size": 3, "nsig": 3},
    "resnet152": {"kernel_size": 3, "nsig": 3},
    "resnext101_64x4d": {"kernel_size": 3, "nsig": 3},
    "swin_v2_s": {"kernel_size": 15, "nsig": 3},
    "vit_l_16": {"kernel_size": 1, "nsig": 3},
    "wide_resnet101_2": {"kernel_size": 3, "nsig": 3},
}
NES_TIM = {
    "convnext_small": {"kernel_size": 15, "nsig": 3},
    "maxvit_t": {"kernel_size": 15, "nsig": 3},
    "resnet152": {"kernel_size": 1, "nsig": 3},
    "resnext101_64x4d": {"kernel_size": 1, "nsig": 3},
    "swin_v2_s": {"kernel_size": 15, "nsig": 3},
    "vit_l_16": {"kernel_size": 1, "nsig": 3},
    "wide_resnet101_2": {"kernel_size": 3, "nsig": 3},
}
ADABELIEF_TIM = {
    "convnext_small": {"kernel_size": 15, "nsig": 3},
    "maxvit_t": {"kernel_size": 3, "nsig": 3},
    "resnet152": {"kernel_size": 3, "nsig": 3},
    "resnext101_64x4d": {"kernel_size": 1, "nsig": 3},
    "swin_v2_s": {"kernel_size": 15, "nsig": 3},
    "vit_l_16": {"kernel_size": 1, "nsig": 3},
    "wide_resnet101_2": {"kernel_size":3, "nsig": 3},
}
ADAM_TIM = {
    "convnext_small": {"kernel_size": 15, "nsig": 3},
    "maxvit_t": {"kernel_size": 3, "nsig": 3},
    "resnet152": {"kernel_size": 1, "nsig": 3},
    "resnext101_64x4d": {"kernel_size": 1, "nsig": 3},
    "swin_v2_s": {"kernel_size": 15, "nsig": 3},
    "vit_l_16": {"kernel_size": 1, "nsig": 3},
    "wide_resnet101_2": {"kernel_size": 3, "nsig": 3},
}
NADAM_TIM = {
    "convnext_small": {"kernel_size": 15, "nsig": 3},
    "maxvit_t": {"kernel_size": 3, "nsig": 3},
    "resnet152": {"kernel_size": 1, "nsig": 3},
    "resnext101_64x4d": {"kernel_size": 1, "nsig": 3},
    "swin_v2_s": {"kernel_size": 15, "nsig": 3},
    "vit_l_16": {"kernel_size": 1, "nsig": 3},
    "wide_resnet101_2": {"kernel_size": 1, "nsig": 3},
}

# SIM
num_samples = 1
eta = 0

def gamma(num):
    return [1 / 2 ** idx for idx in range(num)]

MOM_ADMIX = {
    "convnext_small": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "maxvit_t": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
    "resnet152": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "resnext101_64x4d": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "swin_v2_s": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "vit_l_16": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
    "wide_resnet101_2": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
}
NES_ADMIX = {
    "convnext_small": {"num_samples": num_samples, "gamma": gamma(6), "eta": eta},
    "maxvit_t": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "resnet152": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "resnext101_64x4d": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "swin_v2_s": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "vit_l_16": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "wide_resnet101_2": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
}
ADAM_ADMIX = {
    "convnext_small": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "maxvit_t": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "resnet152": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "resnext101_64x4d": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "swin_v2_s": {"num_samples": num_samples, "gamma": gamma(2), "eta": eta},
    "vit_l_16": {"num_samples": num_samples, "gamma": gamma(6), "eta": eta},
    "wide_resnet101_2": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
}
NADAM_ADMIX = {
    "convnext_small": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
    "maxvit_t": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "resnet152": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "resnext101_64x4d": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "swin_v2_s": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
    "vit_l_16": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
    "wide_resnet101_2": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
}
ADABELIEF_ADMIX = {
    "convnext_small": {"num_samples": num_samples, "gamma": gamma(6), "eta": eta},
    "maxvit_t": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "resnet152": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "resnext101_64x4d": {"num_samples": num_samples, "gamma": gamma(5), "eta": eta},
    "swin_v2_s": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
    "vit_l_16": {"num_samples": num_samples, "gamma": gamma(7), "eta": eta},
    "wide_resnet101_2": {"num_samples": num_samples, "gamma": gamma(8), "eta": eta},
}

# ADMIX
# MOM_ADMIX = {
#     "convnext_small": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "maxvit_t": {"num_samples": 8, "gamma": 0, "eta": 0.5},
#     "resnet152": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "resnext101_64x4d": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "swin_v2_s": {"num_samples": 8, "gamma": 0, "eta": 0.1},
#     "vit_l_16": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "wide_resnet101_2": {"num_samples": 3, "gamma": 0, "eta": 0.2},
# }
# NES_ADMIX = {
#     "convnext_small": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "maxvit_t": {"num_samples": 8, "gamma": 0, "eta": 0.1},
#     "resnet152": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "resnext101_64x4d": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "swin_v2_s": {"num_samples": 8, "gamma": 0, "eta": 0.1},
#     "vit_l_16": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "wide_resnet101_2": {"num_samples": 3, "gamma": 0, "eta": 0.3},
# }
# ADAM_ADMIX = {
#     "convnext_small": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "maxvit_t": {"num_samples": 8, "gamma": 0, "eta": 0.5},
#     "resnet152": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "resnext101_64x4d": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "swin_v2_s": {"num_samples": 8, "gamma": 0, "eta": 0.1},
#     "vit_l_16": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "wide_resnet101_2": {"num_samples": 3, "gamma": 0, "eta": 0.2},
# }
# NADAM_ADMIX = {
#     "convnext_small": {"num_samples": 5, "gamma": 0, "eta": 0.2},
#     "maxvit_t": {"num_samples": 8, "gamma": 0, "eta": 0.1},
#     "resnet152": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "resnext101_64x4d": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "swin_v2_s": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "vit_l_16": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "wide_resnet101_2": {"num_samples": 7, "gamma": 0, "eta": 0.3},
# }
# ADABELIEF_ADMIX = {
#     "convnext_small": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "maxvit_t": {"num_samples": 8, "gamma": 0, "eta": 0.5},
#     "resnet152": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "resnext101_64x4d": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "swin_v2_s": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "vit_l_16": {"num_samples": 3, "gamma": 0, "eta": 0.2},
#     "wide_resnet101_2": {"num_samples": 3, "gamma": 0, "eta": 0.2},
# }

MOM_VT = {
    "convnext_small": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "maxvit_t": {"num_samples": 4, "bound": 1, "dist": "normal"},
    "resnet152": {"num_samples": 4, "bound": .5, "dist": "uniform"},
    "resnext101_64x4d": {"num_samples": 8, "bound": .5, "dist": "normal"},
    "swin_v2_s": {"num_samples": 4, "bound": 1.5, "dist": "normal"},
    "vit_l_16": {"num_samples": 8, "bound": 1.0, "dist": "normal"},
    "wide_resnet101_2": {"num_samples": 8, "bound": 2.0, "dist": "uniform"},
}
NES_VT = {
    "convnext_small": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "maxvit_t": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "resnet152": {"num_samples": 4, "bound": 1.5, "dist": "normal"},
    "resnext101_64x4d": {"num_samples": 8, "bound": 2, "dist": "normal"},
    "swin_v2_s": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "vit_l_16": {"num_samples": 8, "bound": 2.0, "dist": "normal"},
    "wide_resnet101_2": {"num_samples": 4, "bound": 2.0, "dist": "normal"},
}
ADAM_VT = {
    "convnext_small": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "maxvit_t": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "resnet152": {"num_samples": 4, "bound": .5, "dist": "uniform"},
    "resnext101_64x4d": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "swin_v2_s": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "vit_l_16": {"num_samples": 8, "bound": 3.0, "dist": "normal"},
    "wide_resnet101_2": {"num_samples": 4, "bound": .5, "dist": "uniform"},
}
NADAM_VT = {
    "convnext_small": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "maxvit_t": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "resnet152": {"num_samples": 8, "bound": .5, "dist": "uniform"},
    "resnext101_64x4d": {"num_samples": 4, "bound": 2, "dist": "uniform"},
    "swin_v2_s": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "vit_l_16": {"num_samples": 8, "bound": .5, "dist": "normal"},
    "wide_resnet101_2": {"num_samples": 8, "bound": 2.0, "dist": "uniform"},
}
ADABELIEF_VT = {
    "convnext_small": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "maxvit_t": {"num_samples": 4, "bound": 1, "dist": "normal"},
    "resnet152": {"num_samples": 4, "bound": .5, "dist": "uniform"},
    "resnext101_64x4d": {"num_samples": 4, "bound": .5, "dist": "uniform"},
    "swin_v2_s": {"num_samples": 4, "bound": .5, "dist": "normal"},
    "vit_l_16": {"num_samples": 8, "bound": 1.0, "dist": "normal"},
    "wide_resnet101_2": {"num_samples": 8, "bound": .5, "dist": "uniform"},
}

In [81]:
ACT = "linear"
# ACT = "softsign"
# ACT = "tanh"
# ACT = "sigmoid"
# ACT = "sign"

In [82]:
MOMENTUM_HP = {
    "convnext_small": {"beta": 1., "activation": ACT},
    "maxvit_t": {"beta": 1., "activation": ACT},
    "resnet152": {"beta": 0.8, "activation": ACT},
    "resnext101_64x4d": {"beta": 1., "activation": ACT},
    "swin_v2_s": {"beta": 1.0, "activation": ACT},
    "vit_l_16": {"beta": 1., "activation": ACT},
    "wide_resnet101_2": {"beta": .9, "activation": ACT},
}
NESTEROV_HP = {
    "convnext_small": {"beta": 1., "activation": ACT},
    "maxvit_t": {"beta": 1.0, "activation": ACT},
    "resnet152": {"beta": 0.9, "activation": ACT},
    "resnext101_64x4d": {"beta": 1., "activation": ACT},
    "swin_v2_s": {"beta": 1.0, "activation": ACT},
    "vit_l_16": {"beta": 1.0, "activation": ACT},
    "wide_resnet101_2": {"beta": 1., "activation": ACT},
}
ADAM_HP = {
    "convnext_small": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "maxvit_t": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "resnet152": {"beta_1": 0.7, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "resnext101_64x4d": {"beta_1": 0.8, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "swin_v2_s": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "vit_l_16": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "wide_resnet101_2": {"beta_1": 0.7, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
}
NADAM_HP = {
    "convnext_small": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "maxvit_t": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "resnet152": {"beta_1": 0.8, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "resnext101_64x4d": {"beta_1": 0.9, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "swin_v2_s": {"beta_1": 0.99, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "vit_l_16": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "wide_resnet101_2": {"beta_1": 0.9, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
}
ADABELIEF_HP = {
    "convnext_small": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "maxvit_t": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "resnet152": {"beta_1": 0.7, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "resnext101_64x4d": {"beta_1": 0.9, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "swin_v2_s": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "vit_l_16": {"beta_1": 0.999, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
    "wide_resnet101_2": {"beta_1": 0.7, "beta_2": 0.999, "delta": 1e-8, "activation": ACT},
}



import torch
import os
from datetime import datetime



TIME = datetime.now().strftime("%Y/%m/%d-%H:%M:%S")
# SAVE_DIR = rf'/kaggle/working/{TIME.replace("/", "_").replace(":", "_")}/non_targeted_white_box'
# INPUT_DIR = "/kaggle/input/nips-2017-adversarial-learning-development-set/images"
# INPUT_META = "/kaggle/input/thesis-common/common.csv"



SAVE_DIR = os.getcwd() + f'/{TIME.replace("/", "_").replace(":", "_")}'
INPUT_DIR = r"C:\Users\User\Desktop\Thesis\dataset\images"
INPUT_META = r"C:\Users\User\Desktop\Thesis\common.csv"
os.makedirs(SAVE_DIR, exist_ok=True)

BATCH_SIZE = 1
NUM_FIG = 795
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPS = 10
ALPHA = 1
N_ITER = 20
TARGETED = False
MODE = "test"
SAVE_ADV = True
BLACK_BOX_TEST = False
DIM_ACT = False
TIM_ACT = False
ADMIX_ACT = False
VT_ACT = False
ENABLE_AMP = False

# Activation functions

In [83]:
import torch
def softsign(x):
    return x / (1 + x.abs())

def sign(x):
    return x.sign()

def tanh(x):
    return x.tanh()

def sigmoid(x):
    return 2 * x.sigmoid() - 1

def clipped_linear(x):
    return torch.clamp(x, -1, 1)

ACTIVATION = {
    "sign":sign,
    "softsign":softsign,
    "tanh":tanh,
    "sigmoid":sigmoid,
    "linear":clipped_linear,
}

# Utils

In [84]:
import tqdm



import torch



import json, os



from piq.ssim import ssim 



from piq.fsim import fsim



from piq.psnr import psnr



from piq.vif import vif_p



from piq.ms_ssim import multi_scale_ssim



from piq.iw_ssim import information_weighted_ssim



from piq.mdsi import mdsi



from prettytable import PrettyTable







def set_seed(seed):



    torch.manual_seed(seed)



    torch.cuda.manual_seed(seed)



    torch.cuda.manual_seed_all(seed)



    np.random.seed(seed) 



    random.seed(seed)  



    torch.backends.cudnn.benchmark = False



    torch.backends.cudnn.deterministic = True







def calculate_accuracy(model, data_loader):







        model = model.to("cuda")







        model.eval()







        correct = 0







        total = 0







        correct_classification = []















        # with torch.no_grad():







        for path, images, labels in tqdm.tqdm(data_loader):















            images = images.to("cuda")







            labels = labels.to("cuda")  







            outputs = model(images)







            _, predicted = torch.max(outputs.data, 1) 















            total += labels.size(0)







            correct += (predicted == labels).sum().item()  







            for p, q, l in zip(path, predicted, labels):







                 if q == l:







                    correct_classification.append(p)







            







        accuracy = 100 * correct / total







        return accuracy, correct_classification















def attack_metadata(save_dir, time, amp, batch_size, alpha, eps, n_iter, act, num_fig,



                    performed_attck, targeted, mode, black_box,



                    mi_fgsm=None, ni_fgsm=None, agi_fgsm=None, adi_fgsm=None,



                    rmsi_fgsm=None, ai_fgsm=None, ani_fgsm=None, nai_fgsm=None,



                    yogi_fgsm=None, adai_fgsm=None, abi_fgsm=None,



                    dim=None, tim=None, admix=None, vt=None):



    metadata = {



        "Time":time,



        "Enable_amp":amp,



        "Hyperparameters":{"batch_size":batch_size, "alpha":alpha, "eps":eps, "n_iter":n_iter,



                        "activation":act, "num_figure":num_fig},



        "Mode":mode, "Black-Box":black_box,



        "Attack":performed_attck,



        "Targeted":targeted,



        "DIM":dim,



        "TIM":tim,



        "ADMIX":admix,



        "VT":vt,



        "MI-FGSM":mi_fgsm,



        "NI-FGSN":ni_fgsm,



        "AGI-FGSM":agi_fgsm,



        "ADI-FGSM":adi_fgsm,



        "RMSI-FGSM":rmsi_fgsm,



        "AI-FGSM":ai_fgsm,



        "ANI-FGSM":ani_fgsm,



        "NAI-FGSM":nai_fgsm,



        "YOGI":yogi_fgsm, 



        "ADAI":adai_fgsm,



        "ADABELIEF":abi_fgsm,



    }



    time = time.replace("/", "_")



    time = time.replace(":", "_")



    with open(f'{save_dir}/metadata.json', 'w') as file:



        json.dump(metadata, file, indent=4)



    print(metadata["Hyperparameters"], metadata["Targeted"])



    return json.dumps(metadata, indent=4)















def compute_metrics(img, adv_img, verbose=False):







    '''







    return: ssim, psnr, fsim, iw-ssim, ms-ssim, mdsi, vifp, mse, mae







    '''







    _ssim = ssim(img, adv_img, data_range=1.0).item()







    _psnr = psnr(img, adv_img, data_range=1.0).item()







    _fsim = fsim(img, adv_img, data_range=1.0).item()







    _iw_ssim = information_weighted_ssim(img, adv_img, data_range=1.0).item()







    _ms_ssim = multi_scale_ssim(img, adv_img, data_range=1.0).item()







    _mdsi = mdsi(img, adv_img, data_range=1.0).item()







    _vifp = vif_p(img, adv_img, data_range=1.0).item()







    _mse = float(torch.mean((img - adv_img) ** 2).cpu())







    _mae = float(torch.mean(torch.abs(img - adv_img)).cpu())







    if verbose: 







        print(f'SSIM: {_ssim}')







        print(f'PSNR: {_psnr}')







        print(f'FSIM: {_fsim}')







        print(f'IW-SSIM: {_iw_ssim}')







        print(f'MS-SSIM: {_ms_ssim}')







        print(f'MDSI: {_mdsi}')







        print(f'VIFp: {_vifp}')







        print(f'MSE: {_mse}')







        print(f'MAE: {_mae}')







    return [_ssim, _psnr, _fsim, _iw_ssim, _ms_ssim, _mdsi, _vifp, _mse, _mae]















def display_result(m, att, targeted, metrics, metrics_average):







    t = PrettyTable(["ITEM", "VALUE"])







    t.add_row(['METHOD', f"{m}_{att}"])







    t.add_row(["TARGETED", targeted])







    for idx in range(len(metrics)):







        t.add_row([metrics[idx], metrics_average[idx]])







    print(t)      















def read_metadata(dir):







    files = os.listdir(dir)    







    json_file = None







    for file in files:







        if file.endswith('.json'):







            json_file = file







            break    







    if json_file is None:







        raise FileNotFoundError("No metadata JSON file found in the directory")    







    json_path = os.path.join(dir, json_file)







    with open(json_path, 'r') as f:







        metadata = json.load(f)







    return metadata

In [85]:
def black_box(models, model_names, transforms, data):



    acc, success = [], []




    misclassified = np.zeros((len(model_names), BATCH_SIZE))

    misclassified_images = [[] for _ in model_names]



    for idx, m in enumerate(model_names):



        t = transforms[m]



        model = models[m]



        path, adv_image, label, target_label = data



        adv_image = t(adv_image)



        outputs = model(adv_image)



        predicted = outputs.max(1)[1]



        acc.append((label == predicted.cpu()).sum().item())



        for jdx in range(label.size(0)):

            if label[jdx] != predicted.cpu()[jdx]:

                misclassified[idx, jdx] = 1



        misclassified_images[idx].extend([path[i] for i in range(len(path)) if label[i] != predicted[i].cpu()])

        success.append((target_label == predicted.cpu()).sum().item())



    for idx in range(len(misclassified_images)):

        misclassified_images[idx] = "\n".join(misclassified_images[idx])



    return acc, misclassified, misclassified_images, success

In [ ]:
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd, numpy as np
import os, tqdm, torch
import tifffile as tiff
import random
from torch import nn



def main(model_names):
    # Mean and Std of training set
    STD = [0.229, 0.224, 0.225]
    MEAN = [0.485, 0.456, 0.406]
    inv_transform = transforms.Compose([
        transforms.Normalize(
            mean=[-m / s for m, s in zip(MEAN, STD)],
            std=[1 / s for s in STD]        
        )
    ])

    # Data collection for further analysis
    results = {
        "MODEL":[], "CATEGORY":[], "SSIM":[], "PSNR":[], 
        "FSIM":[], "IW-SSIM":[], "MS-SSIM":[], "MDSI":[],
        "VIF_P":[], "MSE":[], "MAE":[], 
        "STEP":[], "LOSS":[], "NOISE":[],
        "SUCCESS":[], "FAIL":[],
        "TRUTH":[], "TARGET":[], "PREDICTED":[]
    }
    metrics = list(results.keys())[2:15]

    INTER_DIR = "targeted" if TARGETED else "non_targeted"
    INDEX = []
    print("DIM:", DIM_ACT, "TIM:", TIM_ACT, "Admix:", ADMIX_ACT, "VT:", VT_ACT)
    # Perform white box attack to each models
    # Veryyy time-consuming
    ACC, SUCCESS, MISCLASSIFIED = [], [], []
    MISCLASSIFIED_IMAGES = []
    BB_SSIM, BB_PSNR = [], []
    INDEX_BLACK_BOX, CATEGORY, WHITE_BOX, BLACK_BOX = [], [], [], []

    models_ = {}
    transforms_ = {}

    for m in model_names:
        model = models.get_model(m, weights="DEFAULT")
        model = nn.DataParallel(model)
        model.to("cuda").eval()
        weights = models.get_model_weights(m).DEFAULT
        transform = weights.transforms()
        models_[m] = model
        transforms_[m] = transform

    for idx, m in enumerate(model_names):
        model = models_[m]
        transform = transforms_[m]
        dataset = ImageCSVDataset(INPUT_META, INPUT_DIR, NUM_FIG, transform, mode=MODE)
        dataloader = DataLoader(dataset, shuffle=False, pin_memory=True, batch_size=BATCH_SIZE)

        HP = [ALPHA, EPS, MEAN, STD, N_ITER, DEVICE]
        attacks = {
            "Momentum": Momentum(
                *HP, 
                tim=MOM_TIM[m], 
                dim=MOM_DIM[m], 
                **MOMENTUM_HP[m], 
                admix=MOM_ADMIX[m] if ADMIX_ACT else None,
                vt=MOM_VT[m] if VT_ACT else None
            ),
            "Nesterov": Nesterov(
                *HP, 
                tim=NES_TIM[m], 
                dim=NES_DIM[m], 
                **NESTEROV_HP[m], 
                admix=NES_ADMIX[m] if ADMIX_ACT else None,  
                vt=NES_VT[m] if VT_ACT else None           
            ),
            "Adam": Adam(
                *HP, 
                tim=ADAM_TIM[m], 
                dim=ADAM_DIM[m], 
                **ADAM_HP[m], 
                admix=ADAM_ADMIX[m] if ADMIX_ACT else None, 
                vt=ADAM_VT[m] if VT_ACT else None           
            ),
            "NAdam": NAdam(
                *HP, 
                tim=NADAM_TIM[m], 
                dim=NADAM_DIM[m], 
                **NADAM_HP[m], 
                admix=NADAM_ADMIX[m] if ADMIX_ACT else None, 
                vt=NADAM_VT[m] if VT_ACT else None           
            ),
            "AdaBelief": AdaBelief(
                *HP, 
                tim=ADABELIEF_TIM[m], 
                dim=ADABELIEF_DIM[m], 
                **ADABELIEF_HP[m], 
                admix=ADABELIEF_ADMIX[m] if ADMIX_ACT else None, 
                vt=ADABELIEF_VT[m] if VT_ACT else None           
            ),
        }


        if ACT != "sign":
            pass
            # attacks["AdaDelta"] = AdaDelta(*HP, **ADADELTA_HP[m], **MISC),
            # attacks["RMSprop"] = AdaDelta(*HP, **RMSPROP_HP[m], **MISC),


        performed_attack = list(attacks.keys())



        attack_metadata(SAVE_DIR, TIME, ENABLE_AMP, BATCH_SIZE, ALPHA, EPS, N_ITER, ACT, NUM_FIG,
            performed_attack, TARGETED, MODE, BLACK_BOX_TEST,
            MOMENTUM_HP if "Momentum" in performed_attack else None,
            NESTEROV_HP if "Nesterov" in performed_attack else None,
            None, None, None,
            ADAM_HP if "Adam" in performed_attack else None,
            None,
            NADAM_HP if "NAdam" in performed_attack else None,
            None, None,
            ADABELIEF_HP if "AdaBelief" in performed_attack else None,
            # Since the transforms object is not JSON serializable
            # None,
            # {k: {key: val for key, val in v.items() if key != "transform"} for k, v in DIM.items()} if DIM_ACT else None,
            # TIM if TIM_ACT else None,
            # ADMIX if ADMIX_ACT else None,
            # VT_ if VT_ACT else None
            None, None, None, None
        )

        for jdx, att in enumerate(attacks):
            set_seed(226)
            ssim_, psnr_ = np.zeros(len(model_names)), np.zeros(len(model_names))
            acc, success = np.zeros(len(model_names)), np.zeros(len(model_names))
            misclassified = np.zeros(len(model_names))
            misclassified_images = ["" for _ in model_names]
            metrics_ave = np.zeros(len(metrics))
            fails, truths, predictions, targets = [], [], [], []
            os.makedirs(f"{SAVE_DIR}/{INTER_DIR}_{m}/{att}", exist_ok=True)
            progress = tqdm.tqdm(dataloader, desc=f"{m}_{att}")
            for path, image, label, target_label in progress:
                # Mixed precision may cause different result
                with torch.autocast(device_type="cuda", dtype=torch.float32, enabled=ENABLE_AMP):
                    adv, predicted, l, perturb, n = attacks[att].forward(model, image, label, 
                                                                        target_label if TARGETED else None)
                predicted = predicted.cpu()
                metrics_ave[12] += (label != predicted).sum().item() if not TARGETED else (target_label == predicted).sum().item()

                if BLACK_BOX_TEST:
                    adv_img = inv_transform(adv)
                    acc_, mis_, mis_images_, success_ = black_box(models_, model_names, transforms_, [path, adv_img, label, target_label])
                    acc += acc_
                    success += success_
                    misclassified += np.sum(mis_, axis=1)
                    for pdx in range(len(misclassified_images)):
                        misclassified_images[pdx] += ("\n" + mis_images_[pdx])
                for kdx in range(image.size(0)):
                    adv_img = inv_transform(adv[kdx]).clamp(0, 1)
                    if SAVE_ADV:
                        adv_img_t = adv_img.cpu().numpy().transpose(1, 2, 0)
                        tiff.imwrite(f"{SAVE_DIR}/{INTER_DIR}_{m}/{att}/{os.path.basename(path[kdx]).replace('.png', '.tiff')}", adv_img_t)
                    if ((not TARGETED) and (label[kdx] == predicted[kdx]) or (TARGETED and (target_label[kdx] != predicted[kdx]))): 
                        fails.append(os.path.basename(path[kdx]))
                        truths.append(str(label[kdx].item()))
                        predictions.append(str(predicted[kdx].item()))
                        if TARGETED:
                            targets.append(str(target_label[kdx].item()))
                        print(path[kdx], n, predicted[kdx], target_label[kdx], label[kdx])
                    elif (TARGETED and (target_label[kdx] == predicted[kdx])) or ((not TARGETED) and (label[kdx] != predicted[kdx])):
                        perturb = (inv_transform(perturb.unsqueeze(0)).cpu())
                        metrics_ave[9] += n # Step
                        metrics_ave[10] += l    # Loss    
                        metrics_ave[11] += torch.norm(perturb, 2).mean().item() # Noise
                        img = inv_transform(image[kdx])
                        img = img.unsqueeze(0).to("cuda")
                        adv_img = adv_img.unsqueeze(0)
                        computed_values = compute_metrics(img, adv_img)

                        if BLACK_BOX_TEST:
                            for idx in range(len(model_names)):
                                    if mis_[idx, kdx]:
                                        ssim_[idx] += computed_values[0]
                                        psnr_[idx] += computed_values[1]
                        for pdx in range(len(computed_values)):
                            metrics_ave[pdx] += computed_values[pdx]

                progress.set_description(f"{m}_{att}/{metrics_ave[-1]}")

            ssim_ /= misclassified
            psnr_ /= misclassified
            metrics_ave[:-1] /= metrics_ave[-1]

            for pdx, key in enumerate(metrics):
                results[key].append(metrics_ave[pdx])

            if BLACK_BOX_TEST:
                # Checkpoint for Black-Box Attack
                INDEX_BLACK_BOX.extend([f"{m}_{att}"] * len(model_names))
                CATEGORY.extend([att] * len(model_names))
                WHITE_BOX.extend([m] * len(model_names))
                BLACK_BOX.extend(model_names)
                ACC.extend(acc)
                BB_PSNR.extend(psnr_)
                BB_SSIM.extend(ssim_)
                MISCLASSIFIED.extend(misclassified)
                MISCLASSIFIED_IMAGES.extend(misclassified_images)
                SUCCESS.extend(success)
                pd.DataFrame({
                    "CATEGORY":CATEGORY, "WHITE_BOX":WHITE_BOX, "BLACK_BOX":BLACK_BOX, 
                    "SSIM":BB_SSIM, "PSNR":BB_PSNR,
                    "ACC":ACC, 
                    "MISCLASSIFIED":MISCLASSIFIED, "SUCCESS": SUCCESS,
                    "MISCLASSIFIED_IMAGES":MISCLASSIFIED_IMAGES
                }, index=INDEX_BLACK_BOX).to_csv(f"checkpoint.csv")

            display_result(m, att, TARGETED, metrics, metrics_ave)
            results["CATEGORY"].append(att)
            results["MODEL"].append(m)
            results["FAIL"].append('\n'.join(fails))
            results["TRUTH"].append(','.join(truths))
            results["TARGET"].append(','.join(targets))
            results["PREDICTED"].append(','.join(predictions))
            INDEX.append(f"{m}_{att}")
            # Checkpoint for White-Box Attack
            pd.DataFrame(results, index=INDEX).to_csv(f"{SAVE_DIR}/result.csv")
            print("Checkpoint Saved...")

model_names = [
    "resnet152",
    "convnext_small",
    "resnext101_64x4d",
    "wide_resnet101_2",
    "maxvit_t",
    "swin_v2_s",
    "vit_l_16"
]

main(model_names)

DIM: False TIM: False Admix: False VT: False
{'batch_size': 1, 'alpha': 1, 'eps': 10, 'n_iter': 20, 'activation': 'linear', 'num_figure': 795} False


resnet152_Momentum/1.0:  20%|██        | 1/5 [00:01<00:07,  1.84s/it]

C:\Users\User\Desktop\Thesis\dataset\images\02eb799d147d5f91.png 20 tensor(884) tensor(772) tensor(624)


resnet152_Momentum/1.0:  40%|████      | 2/5 [00:03<00:05,  1.78s/it]

C:\Users\User\Desktop\Thesis\dataset\images\38546e7d7136c2ab.png 20 tensor(599) tensor(113) tensor(599)


...

resnet152_AdaBelief/3.0:  80%|████████  | 4/5 [00:06<00:01,  1.52s/it]

C:\Users\User\Desktop\Thesis\dataset\images\7b6a9bf5b175e3b2.png 20 tensor(591) tensor(598) tensor(557)


resnet152_AdaBelief/4.0: 100%|██████████| 5/5 [00:07<00:00,  1.51s/it]

C:\Users\User\Desktop\Thesis\dataset\images\72f96c6dc5df2de8.png 20 tensor(843) tensor(783) tensor(884)
+----------+---------------------+
|   ITEM   |        VALUE        |
+----------+---------------------+
|  METHOD  | resnet152_AdaBelief |
| TARGETED |        False        |
|   SSIM   |         0.0         |
|   PSNR   |         0.0         |
|   FSIM   |         0.0         |
| IW-SSIM  |         0.0         |
| MS-SSIM  |         0.0         |
|   MDSI   |         0.0         |
|  VIF_P   |         0.0         |
|   MSE    |         0.0         |
|   MAE    |         0.0         |
|   STEP   |         0.0         |
|   LOSS   |         0.0         |
|  NOISE   |         0.0         |
| SUCCESS  |         4.0         |
+----------+---------------------+
Checkpoint Saved...



C:\Users\User\AppData\Local\Temp\ipykernel_14084\3451330725.py:194: RuntimeWarning: invalid value encountered in divide
  ssim_ /= misclassified
C:\Users\User\AppData\Local\Temp\ipykernel_14084\3451330725.py:195: RuntimeWarning: invalid value encountered in divide
  psnr_ /= misclassified


In [87]:
os.getcwd()

'e:\\results\\cv'